In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import sklearn.preprocessing         # For scale function
import sklearn.metrics as metrics    # For for label size

In [2]:
#global data
import importlib
import xpuzzle
importlib.reload(xpuzzle)
import node
importlib.reload(node)
import search
importlib.reload(search)
inputData = np.array([])
goalState=np.array([[1,2,3,4], [5,6,7,0]])

In [3]:
#i/o
#distance = (2-i)+(2-j) if index == -1 else abs(i-(index/self.state.size))+abs(j-(index%self.state.size))
# file.write("\ncost: " + str(moves))
# file.write("\nrunning_time: " + format(time, '.8f'))

def exportData(name, index, data, heu=-1):
    subname = "-h" + str(heu) if heu > -1 else ""
    solution = f"output/{index}_{name}{subname}" + '_solution.txt'
    search = f"output/{index}_{name}{subname}" + '_search.txt'
    file1 = open(solution, 'w')
    file2 = open(search, 'w')
    if isinstance(data[0], node.Node):
        for ele in data[0].solution:
            file1.write(ele)
        file1.write(f"{data[0].gn} {data[2]}")
        for ele in data[1]:
            file2.write(ele)
    else:  
        file1.write(data[0])
        file2.write(data[1])
    file1.close()
    file2.close()
    
def importData(file_name):
    data = []
    file = open(file_name, 'r')
    for line in file:
        data.append([int(x) for x in line.split(" ")])

    data = np.array(data)
    board_len = len(data[0])
    board_side = int(board_len ** 0.5)
    return data

#sample
def generateInput(total, length=8):
    input = []
    for i in range(total):
        input.append(list(np.random.permutation(length)))
    return input

#analysis
def analysis():
    N = 50
    input = generateInput(N)
    ucs_stat = np.zeros(5)
    gbfs0_stat = np.zeros(5)
    gbfs1_stat = np.zeros(5)
    star0_stat = np.zeros(5)
    star1_stat = np.zeros(5)
    for index, e in enumerate(input):
        print(f"***\t\tsample {index}\t\t***")
        t=xpuzzle.XPuzzle(2, 4, input[index]) 
        s=search.Search(t, 30)

        p = s.uniformCost()
        print(p[0])
        if isinstance(p[0], node.Node):
            ucs_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])
        p = s.greedyBFS(0)
        if isinstance(p[0], node.Node):
            gbfs0_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])
        p = s.greedyBFS(1)
        if isinstance(p[0], node.Node):
            gbfs1_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])
        p  = s.aStar(0)
        if isinstance(p[0], node.Node):
            star0_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])
        p  = s.aStar(1)
        if isinstance(p[0], node.Node):
            star1_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])

    return np.array([ucs_stat, gbfs0_stat, gbfs1_stat, star0_stat, star1_stat])

#scaleup
def scaleup(row, col):
    N = 1
    star1_stat = []
    for r in range(2, row):
        for c in range(2, col):
            print(f"***\t\tsample [{r}, {c}]\t\t***")
            input = generateInput(N, r*c)
            t=xpuzzle.XPuzzle(r, c, input[0]) 
            s=search.Search(t, 120*(r*c/10))

            p  = s.aStar(1)
            print(f"{p[0]} time: {p[2]}")
            if isinstance(p[0], node.Node):
                star1_stat.append([r, c, p[2]])
            else:
                star1_stat.append([r, c, -1])
    
    return star1_stat

#main
def process(input):
    for index, e in enumerate(input):
        print(f"***\t\tsample {index}\t\t***")
        t=xpuzzle.XPuzzle(2, 4, input[index]) 
        s=search.Search(t, 30)

        p = s.uniformCost()
        print(p[0])
        exportData("ucs", index, p)

        for heuristic in range(2):
            p = s.greedyBFS(heuristic)
            print(p[0])
            exportData("gbfs", index, p, heuristic)

            p  = s.aStar(heuristic)
            print(p[0])
            exportData("astar", index, p, heuristic)

In [4]:
inputData = importData('samplePuzzles.txt')
print(inputData, end='\n\n')
process(inputData)

[[3 0 1 4 2 6 5 7]
 [6 3 4 7 1 2 5 0]
 [1 0 3 6 5 2 7 4]
 [0 3 5 1 7 2 4 6]
 [1 3 5 7 0 4 6 2]]

***		sample 0		***
LDCLDLLUZD
RDLULZDTRRRTUCLLLZ
LDCLDLLUZD
RDLLCDTRULDRRURDLULLDZ
LDCLDLLUZD
***		sample 1		***
LURTDRRRULDR
CDRURDRULLDLCLLDRR
LURTDRRRULDR
CDRURDRULLDLCLLDRR
LURTDRRRULDR
***		sample 2		***
RRDLULDRR
RRDLULDRR
RRDLULDRR
RRDLULDRR
RRDLULDRR
***		sample 3		***
ZCRRR
ZCRRR
ZCRRR
ZCRRR
ZCRRR
***		sample 4		***
Z
Z
Z
Z
Z


In [164]:
inputData = importData('samplePuzzles.txt')
print(inputData, end='\n\n')
#inputData[4] no solution
index = 4
t=xpuzzle.XPuzzle(2, 4, inputData[index]) 
t.printPuzzle()
s=search.Search(t)

[[3 0 1 4 2 6 5 7]
 [6 3 4 7 1 2 5 0]
 [1 0 3 6 5 2 7 4]
 [0 3 5 1 7 2 4 6]
 [1 3 5 7 0 4 6 1]]

1 3 5 7 
0 4 6 1 



In [165]:
print("A*")
for heuristic in range(2):
    p  = s.aStar(heuristic)
    print(p[0])
    exportData("astar", index, p, heuristic)


A*
no solution
no solution


In [166]:
print("GBFS")
for heuristic in range(2):
    p = s.greedyBFS(heuristic)
    print(p[0])
    exportData("gbfs", index, p, heuristic)


GBFS
no solution
no solution


In [167]:
print("UCS")
p = s.uniformCost()
print(p[0])
exportData("ucs", index, p)

print("\nMoves")
move = str(p[0])
t.parseMoveSequence(move)


UCS
no solution

Moves
1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 



In [23]:
stat1 = analysis()
print(stat1)

***		sample 0		***
RDLLLZCRDRR
***		sample 1		***
no solution
***		sample 2		***
no solution
***		sample 3		***
no solution
***		sample 4		***
RDRULLZLLDLZ
***		sample 5		***
URCRRRULLDRR
***		sample 6		***
TRDRURTRRDR
***		sample 7		***
ULDZLLLZ
***		sample 8		***
no solution
***		sample 9		***
RDRURDCDZ
***		sample 10		***
RULLDRRTURDRR
***		sample 11		***
LURRRDLLURDR
***		sample 12		***
CLULDLUC
***		sample 13		***
URDRRUTDZ
***		sample 14		***
URDLLURRRD
***		sample 15		***
RRRDTURDRR
***		sample 16		***
no solution
***		sample 17		***
LLZLURDTRRR
***		sample 18		***
no solution
***		sample 19		***
no solution
***		sample 20		***
URRDTUZLDR
***		sample 21		***
RDTUCLLURDR
***		sample 22		***
RURRDLLULC
***		sample 23		***
no solution
***		sample 24		***
no solution
***		sample 25		***
no solution
***		sample 26		***
TDZLURTRRRD
***		sample 27		***
RTCTRDLZ
***		sample 28		***
no solution
***		sample 29		***
no solution
***		sample 30		***
no solution
***		sample 31		***
LURRRTDRRR

In [33]:
stat2 = scaleup(6, 8)
print(stat2)

***		sample [2, 2]		***
LC time: 0.000997781753540039
***		sample [2, 3]		***
UTDRULC time: 0.02499556541442871
***		sample [2, 4]		***
UZLDRTRRR time: 0.011995553970336914
***		sample [2, 5]		***
RCRDRRRTRRULDLCLDR time: 3.3487300872802734
***		sample [2, 6]		***
LDLLLULZDLLLLURDLULDCD time: 3.09175705909729
***		sample [2, 7]		***
no solution time: 120
***		sample [3, 2]		***
DRUUCURD time: 0.035003662109375
***		sample [3, 3]		***
DDLLUZULXLZ time: 0.09196639060974121
***		sample [3, 4]		***
ULURDLLUCUTDRRURUX time: 0.13895511627197266
***		sample [3, 5]		***
no solution time: 120
***		sample [3, 6]		***
no solution time: 120
***		sample [3, 7]		***
no solution time: 120
***		sample [4, 2]		***
DDRULDCX time: 0.013597249984741211
***		sample [4, 3]		***
DRULDLDDZCXRUULURDRDD time: 31.63061285018921
***		sample [4, 4]		***
no solution time: 120
***		sample [4, 5]		***
no solution time: 120
***		sample [4, 6]		***
no solution time: 120
***		sample [4, 7]		***
no solution time: 120
***